# 1-2


In [4]:
using JuMP, HiGHS


I = 1:4    # stations
J = 1:8    # jobs: 1=(T1,S1),2=(T1,S2),3=(T1,S3),4=(T2,S1),5=(T2,S2),6=(T3,S1),7=(T3,S2),8=(T3,S3)

p = [
    # j =  1   2   3   4   5   6   7   8
    # T1   T1  T1  T2  T2  T3  T3  T3
    17  20  17  13  19  11  13   9;   # i=1
    18  18  21  18  21  13   9  13;   # i=2
    12  17   9  21  17   9   9  17;   # i=3
    16  14  21  12  15  14  12  17;   # i=4
]

# availability a[i] and setup s[i]
a = [41, 45, 50, 46]
s = [10, 12,  8,  6]

model = Model(HiGHS.Optimizer)

@variable(model, z[j in J, i in I], Bin)   # z[j,i] = 1 if job j on station i
@variable(model, x[i in I], Bin)           # x[i]=1 if station i is used
@variable(model, C >= 0)                   # makespan

@constraint(model, [j in J], sum(z[j,i] for i in I) == 1)

@constraint(model, [i in I], z[1,i] == z[2,i])
@constraint(model, [i in I], z[2,i] == z[3,i])

@constraint(model, [i in I],
    s[i]*x[i] + sum(p[i,j]*z[j,i] for j in J) <= a[i]
)

# Link usage
@constraint(model, [j in J, i in I], z[j,i] <= x[i])

# Makespan definition
@constraint(model, [i in I],
    s[i]*x[i] + sum(p[i,j]*z[j,i] for j in J) <= C
)

@objective(model, Min, C)

set_silent(model)
optimize!(model)

status = termination_status(model)
@show status
println("Min time to complete: $(round(value(C); digits=2)) min")

println("\nJob → Station assignments:")
for j in J, i in I
    if value(z[j,i]) > 0
        println("  Job $j → Station $i")
    end
end

println("\nStation workloads (including setup):")
for i in I
    load = s[i]*value(x[i]) + sum(p[i,j]*value(z[j,i]) for j in J)
    println("  Station $i: $(round(load; digits=2)) min ", 
            "(used=$(value(x[i])>0.5))")
end


status = MathOptInterface.OPTIMAL
Min time to complete: 46.0 min

Job → Station assignments:
  Job 1 → Station 3
  Job 2 → Station 3
  Job 3 → Station 3
  Job 4 → Station 4
  Job 5 → Station 4
  Job 6 → Station 1
  Job 7 → Station 4
  Job 8 → Station 1

Station workloads (including setup):
  Station 1: 30.0 min (used=true)
  Station 2: 0.0 min (used=false)
  Station 3: 46.0 min (used=true)
  Station 4: 45.0 min (used=true)


# 3-2


In [7]:
using JuMP, GLPK

I = 1:8 # districts
p = [40, 30, 35, 20, 15, 50, 45, 60]                # populations (thousands)
d = [
    0  3  4  6  1  9  8 10;
    3  0  5  4  8  6  1  9;
    4  5  0  2  2  3  5  7;
    6  4  2  0  3  2  5  4;
    1  8  2  3  0  2  2  4;
    9  6  3  2  2  0  3  2;
    8  1  5  5  2  3  0  2;
   10  9  7  4  4  2  2  0
]
n = [d[i,j] ≤ 2 ? 1 : 0 for i in I, j in I]         # reachable within 2 s

# Model
model = Model(GLPK.Optimizer)
@variable(model, a[i in I], Bin)                   # auror at i?
@variable(model, c[i in I], Bin)                   # i covered?

@constraint(model, sum(a[j] for j in I) == 3)      # exactly 3 aurors
@constraint(model, [i in I],
    c[i] <= sum(n[i,j] * a[j] for j in I)           # coverage if ≤2 s away
)

@objective(model, Max, sum(p[i] * c[i] for i in I)) # maximize covered population

optimize!(model)

# Results
println("Auror locations: ", [j for j in I if value(a[j]) > 0])
println("Covered districts: ", [i for i in I if value(c[i]) > 0])
println("Total covered population (thousands): ", objective_value(model))


Auror locations: [1, 4, 7]
Covered districts: [1, 2, 3, 4, 5, 6, 7, 8]
Total covered population (thousands): 295.0
